In [2]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [3]:
RAW_PATH = "../data/raw/"
PROCESSED_PATH = "../data/processed/tabular/"
os.makedirs(PROCESSED_PATH, exist_ok=True)

In [4]:
datasets = {
    "supply_chain_resilience": os.path.join(RAW_PATH, "supply_chain_resilience_dataset.csv"),
    "us_supply_chain_risk": os.path.join(RAW_PATH, "data.csv"),
    "cargo2000": os.path.join(RAW_PATH, "c2k_data_comma.csv"),
    "smart_logistics": os.path.join(RAW_PATH, "smart_logistics_dataset.csv"),
    "dynamic_logistics": os.path.join(RAW_PATH, "dynamic_supply_chain_logistics_dataset.csv"),
}

Cleaning Functions

In [9]:
def normalize_columns(df):
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def parse_dates(df, date_columns):
    for c in date_columns:
        df[c] = pd.to_datetime(df[c], errors='coerce')
    return df

def fill_missing(df):
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].fillna("None")  # keep "None" intact
        elif np.issubdtype(df[c].dtype, np.number):
            df[c] = df[c].fillna(df[c].median())
    return df


# 1. Supply Chain Resilience Dataset
df_res = pd.read_csv(datasets["supply_chain_resilience"])
df_res = normalize_columns(df_res)

# Parse dates
df_res = parse_dates(df_res, ["order_date", "dispatch_date", "delivery_date"])

# Feature engineering: lead time
df_res["lead_time_days"] = (df_res["delivery_date"] - df_res["dispatch_date"]).dt.days

# Fill missing if any
df_res = fill_missing(df_res)

# Save
out_path = os.path.join(PROCESSED_PATH, "supply_chain_resilience_clean.csv")
df_res.to_csv(out_path, index=False)
print(f"✅ Saved {out_path}, shape={df_res.shape}")



# 2. US Supply Chain Risk Dataset
df_us = pd.read_csv(datasets["us_supply_chain_risk"])
df_us = normalize_columns(df_us)

# Parse dates
df_us = parse_dates(df_us, ["order_date", "dispatch_date", "delivery_date"])

# Feature: lead time
df_us["lead_time_days"] = (df_us["delivery_date"] - df_us["dispatch_date"]).dt.days

df_us = fill_missing(df_us)

out_path = os.path.join(PROCESSED_PATH, "us_supply_chain_risk_clean.csv")
df_us.to_csv(out_path, index=False)
print(f"✅ Saved {out_path}, shape={df_us.shape}")



# 3. Cargo2000 Dataset
df_c2k = pd.read_csv(datasets["cargo2000"], low_memory=False)

# Replace '?' with NaN
df_c2k.replace("?", np.nan, inplace=True)

df_c2k = normalize_columns(df_c2k)

# Convert numeric columns to float
for c in df_c2k.columns:
    df_c2k[c] = pd.to_numeric(df_c2k[c], errors="ignore")

# Feature engineering: example delay = effective - planned
planned_cols = [c for c in df_c2k.columns if c.endswith("_p")]
effective_cols = [c for c in df_c2k.columns if c.endswith("_e")]

for p, e in zip(planned_cols, effective_cols):
    delay_col = p.replace("_p", "_delay")
    df_c2k[delay_col] = pd.to_numeric(df_c2k[e], errors="coerce") - pd.to_numeric(df_c2k[p], errors="coerce")

# Fill missing values
df_c2k = fill_missing(df_c2k)

out_path = os.path.join(PROCESSED_PATH, "cargo2000_clean.csv")
df_c2k.to_csv(out_path, index=False)
print(f"✅ Saved {out_path}, shape={df_c2k.shape}")



# 4. Smart Logistics Dataset
df_smart = pd.read_csv(datasets["smart_logistics"])
df_smart = normalize_columns(df_smart)

# Parse timestamp
df_smart = parse_dates(df_smart, ["timestamp"])

# Fill missing
df_smart = fill_missing(df_smart)

out_path = os.path.join(PROCESSED_PATH, "smart_logistics_clean.csv")
df_smart.to_csv(out_path, index=False)
print(f"✅ Saved {out_path}, shape={df_smart.shape}")



# 5. Dynamic Supply Chain Logistics Dataset
df_dyn = pd.read_csv(datasets["dynamic_logistics"])
df_dyn = normalize_columns(df_dyn)

# Parse timestamp
df_dyn = parse_dates(df_dyn, ["timestamp"])

# Fill missing
df_dyn = fill_missing(df_dyn)

out_path = os.path.join(PROCESSED_PATH, "dynamic_logistics_clean.csv")
df_dyn.to_csv(out_path, index=False)
print(f"✅ Saved {out_path}, shape={df_dyn.shape}")


✅ Saved ../data/processed/tabular/supply_chain_resilience_clean.csv, shape=(1000, 25)
✅ Saved ../data/processed/tabular/us_supply_chain_risk_clean.csv, shape=(1000, 25)


C:\Users\AADITYA\AppData\Local\Temp\ipykernel_13952\4208740638.py:67: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_c2k[c] = pd.to_numeric(df_c2k[c], errors="ignore")
C:\Users\AADITYA\AppData\Local\Temp\ipykernel_13952\4208740638.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_c2k[delay_col] = pd.to_numeric(df_c2k[e], errors="coerce") - pd.to_numeric(df_c2k[p], errors="coerce")
C:\Users\AADITYA\AppData\Local\Temp\ipykernel_13952\4208740638.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

✅ Saved ../data/processed/tabular/cargo2000_clean.csv, shape=(3943, 130)
✅ Saved ../data/processed/tabular/smart_logistics_clean.csv, shape=(1000, 16)
✅ Saved ../data/processed/tabular/dynamic_logistics_clean.csv, shape=(32065, 26)


In [10]:
import glob, os

processed_files = glob.glob(PROCESSED_PATH + "*.csv")
print("📊 Processed Datasets:")

for f in processed_files:
    df = pd.read_csv(f, nrows=3)   # preview 3 rows
    print(f"{os.path.basename(f)} → {df.shape}")
    display(df.head(5))

📊 Processed Datasets:
cargo2000_clean.csv → (3, 130)


,nr,i1_legid,i1_rcs_p,i1_rcs_e,i1_dep_1_p,i1_dep_1_e,i1_dep_1_place,i1_rcf_1_p,i1_rcf_1_e,i1_rcf_1_place,...,i3_rcf_3_delay,i3_dlv_delay,o_rcs_delay,o_dep_1_delay,o_rcf_1_delay,o_dep_2_delay,o_rcf_2_delay,o_dep_3_delay,o_rcf_3_delay,o_dlv_delay
0,0.0,5182.0,199.0,218.0,210.0,215.0,609.0,935.0,736.0,256.0,...,-151.5,-2825.0,-2360.0,2625.0,-65.0,141.0,-185.0,94.5,-165.0,-346.0
1,1.0,6523.0,844.0,584.0,90.0,297.0,700.0,1935.0,1415.0,431.0,...,-151.5,-2825.0,-14.0,-14.0,-309.0,141.0,-185.0,94.5,-165.0,-3425.0
2,2.0,5878.0,4380.0,4119.0,90.0,280.0,456.0,905.0,547.0,700.0,...,-151.5,-2825.0,-791.0,729.0,80.0,141.0,-185.0,94.5,-165.0,970.0


dynamic_logistics_clean.csv → (3, 26)


,timestamp,vehicle_gps_latitude,vehicle_gps_longitude,fuel_consumption_rate,eta_variation_hours,traffic_congestion_level,warehouse_inventory_level,loading_unloading_time,handling_equipment_availability,order_fulfillment_status,...,iot_temperature,cargo_condition_status,route_risk_level,customs_clearance_time,driver_behavior_score,fatigue_monitoring_score,disruption_likelihood_score,delay_probability,risk_classification,delivery_time_deviation
0,2021-01-01 00:00:00,40.375568,-77.014318,5.136512,4.998009,5.927586,985.716862,4.951392,0.481294,0.761166,...,0.574400,0.777263,1.182116,0.502006,0.033843,0.978599,0.506152,0.885291,Moderate Risk,9.110682
1,2021-01-01 01:00:00,33.507818,-117.036902,5.101512,0.984929,1.591992,396.700206,1.030379,0.620780,0.196594,...,-9.753493,0.091839,9.611988,0.966774,0.201725,0.918586,0.980784,0.544178,High Risk,8.175281
2,2021-01-01 02:00:00,30.020640,-75.269224,5.090803,4.972665,8.787765,832.408935,4.220229,0.810933,0.152742,...,-6.491034,0.253529,6.570431,0.945627,0.264045,0.394215,0.998633,0.803322,High Risk,1.283594


smart_logistics_clean.csv → (3, 16)


,timestamp,asset_id,latitude,longitude,inventory_level,shipment_status,temperature,humidity,traffic_status,waiting_time,user_transaction_amount,user_purchase_frequency,logistics_delay_reason,asset_utilization,demand_forecast,logistics_delay
0,2024-03-20 00:11:14,Truck_7,-65.7383,11.2497,390,Delayed,27.0,67.8,Detour,38,320,4,NaN,60.1,285,1
1,2024-10-30 07:53:51,Truck_6,22.2748,-131.7086,491,In Transit,22.5,54.3,Heavy,16,439,7,Weather,80.9,174,1
2,2024-07-29 18:42:48,Truck_10,54.9232,79.5455,190,In Transit,25.2,62.2,Detour,34,355,3,NaN,99.2,260,0


supply_chain_resilience_clean.csv → (3, 25)


,order_id,buyer_id,supplier_id,product_category,quantity_ordered,order_date,dispatch_date,delivery_date,shipping_mode,order_value_usd,...,organization_id,dominant_buyer_flag,available_historical_records,data_sharing_consent,federated_round,parameter_change_magnitude,communication_cost_mb,energy_consumption_joules,supply_risk_flag,lead_time_days
0,O1000,B33,S23,Textiles,469,2023-10-24,2023-10-27,2023-10-28,Rail,36273.99,...,Org13,0,127,1,10,0.0225,2.27,236.06,0,1
1,O1001,B1,S20,Machinery,365,2023-07-07,2023-07-08,2023-07-09,Road,34780.36,...,Org8,1,909,1,7,0.0412,3.10,257.80,0,1
2,O1002,B2,S10,Food,333,2023-12-28,2023-12-29,2024-01-07,Rail,7154.54,...,Org4,0,262,1,8,0.1183,2.82,165.38,1,9


us_supply_chain_risk_clean.csv → (3, 25)


,order_id,buyer_id,supplier_id,product_category,quantity_ordered,order_date,dispatch_date,delivery_date,shipping_mode,order_value_usd,...,organization_id,dominant_buyer_flag,available_historical_records,data_sharing_consent,federated_round,parameter_change_magnitude,communication_cost_mb,energy_consumption_joules,supply_risk_flag,lead_time_days
0,O1000,B33,S23,Textiles,469,2023-10-24,2023-10-27,2023-10-28,Rail,36273.99,...,Org13,0,127,1,10,0.0225,2.27,236.06,0,1
1,O1001,B1,S20,Machinery,365,2023-07-07,2023-07-08,2023-07-09,Road,34780.36,...,Org8,1,909,1,7,0.0412,3.10,257.80,0,1
2,O1002,B2,S10,Food,333,2023-12-28,2023-12-29,2024-01-07,Rail,7154.54,...,Org4,0,262,1,8,0.1183,2.82,165.38,1,9
